In [7]:
loadModel(Modelica) 

True

In [8]:
model cylinder "
+ When using the units packages (eg SIunits), the types specify immutable units.
  You have to use the Conversions subpackages to have values expressed in thems of other units.
+ displayUnit may be ignored by the modeling tool. OMPython in Jupyter Notebook 
  seems to ignore displayUnit.  
"

  import Modelica.Constants.*;
//   import Modelica.SIunits.{Angle,AngularVelocity,Length,Diameter,Volume,Pressure,Temperature};
  import Modelica.SIunits.*;
//   import Modelica.SIunits.Conversions.NonSIunits.{Angle_deg,AngularVelocity_rpm};
  import Modelica.SIunits.Conversions.NonSIunits.*;
//   import Modelica.SIunits.Conversions.{from_deg,from_rpm}; 
  import Modelica.SIunits.Conversions.*; 
  import Modelica.Media.*;

//   type Volume = Real(final quantity = "Volume", final unit = "l");

     
  parameter AngularVelocity_rpm N_rpm=600 "engine speed [rpm]";
  parameter Diameter bore(displayUnit="mm")=0.080 "bore [m]";
  parameter Length stroke(displayUnit="mm")=0.090 "stroke [m]";
  parameter Length conrod(displayUnit="mm")=0.140 "connecting rod length [m]";
  parameter Pressure P_bdc(displayUnit="bar")=100000 "pressure at BDC [Pa]";
  parameter Temperature T_bdc=300 "temp at BDC [K]";
  parameter Real n=1.3 "polytropic exponent";
  parameter Real rc=12.5 "compression ratio";
  parameter Volume Vd(displayUnit="l") = pi/4*bore^2*stroke "displacement";  
  parameter Volume V_tdc(displayUnit="l") = Vd/(rc-1) "clearance (min) volume";
  parameter Volume V_bdc(displayUnit="l") = V_tdc*rc "max volume";  
  parameter SpecificHeatCapacity R(displayUnit="J/(kg.K)") = 8314/44 "mass gas constant";
  parameter Mass m_cyl(displayUnit="kg") = P_bdc * V_bdc / (R * T_bdc) "cylinder charge mass";
//   parameter Density rho "cylinder charge density";
    
  package CO2 = Modelica.Media.IdealGases.SingleGases.CO2; 


// variables
  Angle_deg degCA(start=0) "crank angle [deg]. Zero degrees is TDC";
  Angle_deg degCAmod360 "crank angle [deg]";
  Angle CA "crank angle [rad]";
  Angle theta "conrod angle from cylinder axis [rad]";
  AngularVelocity N "engine speed [rad/s]";
  Volume V_cyl(displayUnit = "l") "instantaneous cylinder volume";
  Pressure P_cyl(displayUnit="bar") "cylinder pressure [Pa]";
  Temperature T_cyl "cylinder temperature [K]";
  Temperature T_ig "cylinder temperature [K]";
  Length x(displayUnit="mm") "piston relative distance from TDC";
  CO2.BaseProperties medium;  
  Density rho_cyl "cylinder charge density";
  Volume_litre V_cyl_litre "instantaneous cylinder volume";

  
// initial equation
//  degCA = 90;
   
equation
// define crank angle; display it in degrees, calculate it always in radians 
  CA = from_deg(degCA);
  N = from_rpm(N_rpm);
  der(CA) = N;
  degCAmod360 = mod(degCA,360);
  //degCA=mod(degCA,720);
  
// slider-crank geometry
  sin(CA)/conrod = sin(theta)/(stroke/2) "Law of sines";
  x = (( conrod+stroke/2 ))
     -(( conrod*cos(theta)+cos(CA)*stroke/2 )) "x=0 @TDC; x=S @BDC";
  V_cyl = V_tdc + x*pi/4*bore^2;
  V_cyl_litre = to_litre(V_cyl);
 
// let's add some thermodynamics
  P_cyl/P_bdc = (V_bdc/V_cyl)^n  "polytropic relationship";
  P_cyl*V_cyl/T_cyl = P_bdc*V_bdc/T_bdc "ideal gas law";
  rho_cyl = m_cyl/V_cyl;
//   state = CO2.setState_dT(rho_cyl, T_cyl);
  medium.p = P_cyl;
  medium.d = rho_cyl;
  T_ig = medium.T;
  
  
end cylinder;


('cylinder',)

In [9]:
simulate(cylinder, stopTime = 0.25)

{'resultFile': '/user/jovyan/work/cylinder_res.mat', 'simulationOptions': "startTime = 0.0, stopTime = 0.25, numberOfIntervals = 500, tolerance = 1e-06, method = 'dassl', fileNamePrefix = 'cylinder', options = '', outputFormat = 'mat', variableFilter = '.*', cflags = '', simflags = ''", 'messages': 'LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.\nLOG_SUCCESS       | info    | The simulation finished successfully.\n', 'timeFrontend': 1.7900488, 'timeBackend': 0.0207607, 'timeSimCode': 0.04738469999999995, 'timeTemplates': 0.08125940000000001, 'timeCompile': 1.7428028, 'timeSimulation': 0.0564405, 'timeTotal': 3.743401}

In [10]:
plot(T_cyl,T_ig)

In [5]:
plot(V_cyl_litre)

In [6]:
plotParametric2(degCAmod360, {degCAmod360})